In [ ]:
import subprocess
import shutil
import torch
import torchvision
from torchvision import transforms

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_data = torchvision.datasets.ImageFolder(root='dataset/train', transform=transform)
test_data = torchvision.datasets.ImageFolder(root='dataset/test', transform=transform)


In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=128, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(dataset=test_data, batch_size=1028, shuffle=True, num_workers=2)

In [ ]:
net = torch.hub.load('pytorch/vision:v0.10.0', 'googlenet', pretrained=True)


In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
test_accs = dict()

for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.cuda(), labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 50 == 49:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 50:.3f}')
            running_loss = 0.0
            
        # for every so and so iterations, check test_accs and save it to dictionary
        # maybe save the weights on that iteration too??
        
        
print('Finished Training')

In [ ]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
confusion_matrix = torch.zeros(nb_classes, nb_classes)
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.cuda(), labels.cuda()
        classes = classes.cuda()
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        for t, p in zip(classes.view(-1), predicted.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

print(f'Accuracy of the network on the test images: {100 * correct // total} %')
print(confusion_matrix.diag()/confusion_matrix.sum(1))